In [1]:
import pickle
import os
import numpy as np
import tensorflow.compat.v1 as tf

tf.disable_v2_behavior()

CIFAR_DIR = './cifar-10-batches-py'
print(os.listdir(CIFAR_DIR))

def loadData(filename):
    with open(filename, "rb") as f:
        data = pickle.load(f, encoding='latin1')
        return data['data'], data['labels']

2023-06-17 23:38:35.966908: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Instructions for updating:
non-resource variables are not supported in the long term
['data_batch_1', 'readme.html', 'batches.meta', 'data_batch_2', 'data_batch_5', 'test_batch', 'data_batch_4', 'data_batch_3']


In [2]:
class CifarData:
    def __init__(self, filenames, isShuffle):
        allData = []
        allLabels = []
        for filename in filenames:
            data, labels = loadData(filename)
            allData.append(data)
            allLabels.append(labels)
        self._data = np.vstack(allData)
        self._data = self._data / 127.5 - 1
        self._labels = np.hstack(allLabels)
        self._size = self._data.shape[0]
        self._startPos = 0
        self._isShuffle = isShuffle
        if isShuffle:
            self.shuffle()
            
    def shuffle(self):
        p = np.random.permutation(self._size)
        self._data = self._data[p]
        self._labels = self._labels[p]

    def nextBatch(self, size):
        endPos = self._startPos + size
        if endPos > self._size:
            if self._isShuffle:
                self.shuffle()
                self._startPos = 0
                endPos = size
            else:
                raise Exception("No more samples")
            
        if endPos > self._size:
            raise Exception("Sample size is less than %d", size)
        
        batch_data = self._data[self._startPos: endPos]
        batch_labels = self._labels[self._startPos: endPos]
        self._startPos = endPos
        return batch_data, batch_labels

train_filenames = [os.path.join(CIFAR_DIR, "data_batch_%d" % i) for i in range(1, 6)]
train_data = CifarData(train_filenames, True)
# print(training_data._size)
# print(training_data.nextBatch(10000))

test_filenames = [os.path.join(CIFAR_DIR, "test_batch")]
test_data = CifarData(test_filenames, False)
# print(test_data._size)
# print(test_data.nextBatch(2000))

In [3]:
def residual_block(x, output_channel):
    input_channel = x.get_shape().as_list()[-1]
    if input_channel * 2 == output_channel:
        increase_dim = True
        strides = (2, 2)
    elif input_channel == output_channel:
        increase_dim = False
        strides = (1, 1)
    else:
        raise Exception("the input channel isn't match")
    
    conv1 = tf.layers.conv2d(x, output_channel, (3, 3), strides = strides, padding = "same", activation = tf.nn.relu, name = "conv1")
    conv2 = tf.layers.conv2d(conv1, output_channel, (3, 3), strides = (1, 1), padding = "same", activation = tf.nn.relu, name = "conv2")
    
    if increase_dim:
        pooled_x = tf.layers.average_pooling2d(x, (2, 2), (2, 2), padding = "valid")
        padded_x = tf.pad(pooled_x, [[0, 0], [0, 0], [0, 0], [input_channel // 2, input_channel // 2]])
    else:
        padded_x = x
        
    return conv2 + padded_x

def resnet(x, num_residual_blocks, num_filter_base, class_num):
    num_stages = len(num_residual_blocks)
    layers = []
    input_size = x.get_shape().as_list()[1:]
    with tf.variable_scope("conv0"):
        conv0 = tf.layers.conv2d(x, num_filter_base, (3, 3), padding = "same", activation = tf.nn.relu, name = "conv0")
        layers.append(conv0)
    for stage in range(num_stages):
        for residual in range(num_residual_blocks[stage]):
            with tf.variable_scope("conv%d-%d" % (stage + 1, residual)):
                block = residual_block(layers[-1], num_filter_base * (2 ** stage))
                layers.append(block)
    multiplier = 2 ** (num_stages - 1)
    assert layers[-1].get_shape().as_list()[1:] == [
        input_size[0] / multiplier,
        input_size[1] / multiplier,
        num_filter_base * multiplier]
    with tf.variable_scope("fc"):
        global_pool = tf.reduce_mean(layers[-1], [1, 2])
        logits = tf.layers.dense(global_pool, class_num)
        layers.append(logits)
    return layers[-1]

x = tf.placeholder(tf.float32, [None, 3072])
y = tf.placeholder(tf.int64, [None])

x_reshape = tf.reshape(x, [-1, 3, 32, 32])
x_reshape = tf.transpose(x_reshape, perm = [0, 2, 3, 1])

y_ = resnet(x_reshape, [2, 3, 2], 32, 10)

# p_y = tf.nn.softmax(y_)
# y_one_hot = tf.one_hot(y, 10, dtype = tf.float32)
# loss = tf.reduce_mean(tf.square(y_one_hot - p_y))
loss = tf.losses.sparse_softmax_cross_entropy(labels = y, logits = y_)

predict = tf.argmax(y_, 1)
correct_prediction = tf.equal(predict, y)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

with (tf.name_scope("train_op")):
    train_op = tf.train.AdamOptimizer(1e-3).minimize(loss)


/var/folders/n4/d8w2s55130vdn6290bjbhblc0000gn/T/ipykernel_7021/1521469276.py:28: UserWarning: `tf.layers.conv2d` is deprecated and will be removed in a future version. Please Use `tf.keras.layers.Conv2D` instead.
  conv0 = tf.layers.conv2d(x, num_filter_base, (3, 3), padding = "same", activation = tf.nn.relu, name = "conv0")
/var/folders/n4/d8w2s55130vdn6290bjbhblc0000gn/T/ipykernel_7021/1521469276.py:12: UserWarning: `tf.layers.conv2d` is deprecated and will be removed in a future version. Please Use `tf.keras.layers.Conv2D` instead.
  conv1 = tf.layers.conv2d(x, output_channel, (3, 3), strides = strides, padding = "same", activation = tf.nn.relu, name = "conv1")
/var/folders/n4/d8w2s55130vdn6290bjbhblc0000gn/T/ipykernel_7021/1521469276.py:13: UserWarning: `tf.layers.conv2d` is deprecated and will be removed in a future version. Please Use `tf.keras.layers.Conv2D` instead.
  conv2 = tf.layers.conv2d(conv1, output_channel, (3, 3), strides = (1, 1), padding = "same", activation = tf.nn

In [ ]:
init = tf.global_variables_initializer()
batch_size = 20
train_steps = 10000
test_steps = 100

with tf.Session() as sess:
    sess.run(init)
    for i in range(train_steps):
        batch_data, batch_labels = train_data.nextBatch(batch_size)
        loss_val, acc_val, _ = sess.run(
            [loss, accuracy, train_op],
            feed_dict = {
                x: batch_data,
                y: batch_labels
            }
        )
        
        if (i + 1) % 500 == 0:
            print("[Train] Step: %d, loss: %4.5f, acc: %4.5f" % (i + 1, loss_val, acc_val))
            
        if (i + 1) % 5000 == 0:
            test_data = CifarData(test_filenames, False)
            all_test_acc_val = []
            for j in range(test_steps):
                test_batch_data, test_batch_labels = test_data.nextBatch(batch_size)
                test_acc_val = sess.run(
                    [accuracy],
                    feed_dict = {
                        x: test_batch_data,
                        y: test_batch_labels
                    }
                )
                all_test_acc_val.append(test_acc_val)
            test_acc_val = np.mean(all_test_acc_val)
            print("[Test ] Step: %d, acc: %4.5f" % (i + 1, test_acc_val))

[Train] Step: 500, loss: 1.90385, acc: 0.15000
[Train] Step: 1000, loss: 1.78979, acc: 0.40000
[Train] Step: 1500, loss: 1.20600, acc: 0.60000
[Train] Step: 2000, loss: 1.47479, acc: 0.40000
[Train] Step: 2500, loss: 1.16510, acc: 0.55000
[Train] Step: 3000, loss: 1.34522, acc: 0.55000
[Train] Step: 3500, loss: 0.99650, acc: 0.60000
[Train] Step: 4000, loss: 0.97915, acc: 0.65000
[Train] Step: 4500, loss: 1.15956, acc: 0.50000
[Train] Step: 5000, loss: 1.10796, acc: 0.60000
[Test ] Step: 5000, acc: 0.64150
